# Lab | BabyAGI with agent

**Change the planner objective below by changing the objective and the associated prompts and potential tools and agents - Wear your creativity and AI engineering hats
You can't get this wrong!**

You would need the OpenAI API KEY and the [SerpAPI KEY](https://serpapi.com/manage-api-keyhttps://serpapi.com/manage-api-key) to run this lab.


## BabyAGI with Tools

This notebook builds on top of [baby agi](baby_agi.html), but shows how you can swap out the execution chain. The previous execution chain was just an LLM which made stuff up. By swapping it out with an agent that has access to tools, we can hopefully get real reliable information

## Install and Import Required Modules

In [1]:
from typing import Optional

from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_experimental.autonomous_agents import BabyAGI
from langchain_openai import OpenAI, OpenAIEmbeddings

## Connect to the Vector Store

Depending on what vectorstore you use, this step may look different.

In [2]:
# %pip install faiss-cpu > /dev/null
# %pip install google-search-results > /dev/null

from langchain.docstore import InMemoryDocstore
from langchain_community.vectorstores import FAISS

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
SERPAPI_API_KEY = os.getenv('SERPAPI_API_KEY')

In [5]:
# Define your embedding model
embeddings_model = OpenAIEmbeddings()

# Initialize the vectorstore as empty
import faiss

embedding_size = 1536

index = faiss.IndexFlatL2(embedding_size)

vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


In [12]:
print(vectorstore)
print(type(vectorstore))

<class 'langchain_community.vectorstores.faiss.FAISS'>


## Define the Chains

BabyAGI relies on three LLM chains:
- Task creation chain to select new tasks to add to the list
- Task prioritization chain to re-prioritize tasks
- Execution Chain to execute the tasks


NOTE: in this notebook, the Execution chain will now be an agent.

In [7]:
from langchain.agents import AgentExecutor, Tool, ZeroShotAgent
from langchain.chains import LLMChain
from langchain_community.utilities import SerpAPIWrapper
from langchain_openai import OpenAI

todo_prompt = PromptTemplate.from_template(
    "You are a planner who is an expert at coming up with a todo list for a given objective. Come up with a todo list for this objective: {objective}"
)

todo_chain = LLMChain(llm = OpenAI(temperature = 0), prompt = todo_prompt)

search = SerpAPIWrapper()

tools = [
    Tool(
        name = "Search",
        func = search.run,
        description = "useful for when you need to answer questions about current events",
    ),
    Tool(
        name = "TODO",
        func = todo_chain.run,
        description = "useful for when you need to come up with todo lists. Input: an objective to create a todo list for. Output: a todo list for that objective. Please be very clear what the objective is!",
    ),
]


prefix = """You are an AI who performs one task based on the following objective: {objective}. Take into account these previously completed tasks: {context}."""

suffix = """Question: {task}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix = prefix,
    suffix = suffix,
    input_variables = ["objective", "task", "context", "agent_scratchpad"],
)

In [ ]:
llm = OpenAI(temperature = 0)

llm_chain = LLMChain(llm = llm, prompt = prompt)

tool_names = [tool.name for tool in tools]

agent = ZeroShotAgent(llm_chain = llm_chain, allowed_tools = tool_names)

agent_executor = AgentExecutor.from_agent_and_tools(
    agent = agent, tools = tools, verbose = True
)

/tmp/ipykernel_114886/3064747660.py:4: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)


In [18]:
print(agent, '\n', type(agent))

'''
llm_chain = LLMChain(
    verbose = False, 

    prompt = PromptTemplate(input_variables = ['agent_scratchpad', 'context', 'objective', 'task'], 
                            input_types = {}, 
                            partial_variables = {}, 
                            template = """You are an AI who performs one task based on the following objective: {objective}. Take into account these previously completed tasks: {context}.\n\n
                            
                            Search(query: str, **kwargs: Any) -> str - useful for when you need to answer questions about current events\n
                            TODO(*args: Any, 
                            callbacks: Union[list[langchain_core.callbacks.base.BaseCallbackHandler], langchain_core.callbacks.base.BaseCallbackManager, NoneType] = None, 
                            tags: Optional[List[str]] = None, 
                            metadata: Optional[Dict[str, Any]] = None, 
                            **kwargs: Any) -> Any - useful for when you need to come up with todo lists. 
                            Input: an objective to create a todo list for. 
                            Output: a todo list for that objective. Please be very clear what the objective is!\n\n
                            
                            Use the following format:\n\n
                            
                            Question: the input question you must answer\n
                            Thought: you should always think about what to do\n
                            Action: the action to take, should be one of [Search, TODO]\n
                            Action Input: the input to the action\n
                            Observation: the result of the action\n
                            ... (this Thought/Action/Action Input/Observation can repeat N times)\n
                            Thought: I now know the final answer\n
                            Final Answer: the final answer to the original input question\n\n
                            
                            Question: {task}\n
                            {agent_scratchpad}"""), 

    llm = OpenAI(client = <openai.resources.completions.Completions object at 0x7f139cb6cf50>, 
                 async_client = <openai.resources.completions.AsyncCompletions object at 0x7f13996efd40>, 
                 temperature = 0.0, 
                 model_kwargs = {}, 
                 openai_api_key = SecretStr('**********')), 

    output_parser = StrOutputParser(), 

    llm_kwargs = {}
    )


output_parser = MRKLOutputParser() 

allowed_tools = ['Search', 'TODO']
'''

llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['agent_scratchpad', 'context', 'objective', 'task'], input_types={}, partial_variables={}, template='You are an AI who performs one task based on the following objective: {objective}. Take into account these previously completed tasks: {context}.\n\nSearch(query: str, **kwargs: Any) -> str - useful for when you need to answer questions about current events\nTODO(*args: Any, callbacks: Union[list[langchain_core.callbacks.base.BaseCallbackHandler], langchain_core.callbacks.base.BaseCallbackManager, NoneType] = None, tags: Optional[List[str]] = None, metadata: Optional[Dict[str, Any]] = None, **kwargs: Any) -> Any - useful for when you need to come up with todo lists. Input: an objective to create a todo list for. Output: a todo list for that objective. Please be very clear what the objective is!\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what 

### Run the BabyAGI

Now it's time to create the BabyAGI controller and watch it try to accomplish your objective.

In [9]:
OBJECTIVE = "Write a weather report for SF today"

In [10]:
# Logging of LLMChains
verbose = False

# If None, will keep on going forever
max_iterations: Optional[int] = 3

baby_agi = BabyAGI.from_llm(
    llm = llm,
    vectorstore = vectorstore,
    task_execution_chain = agent_executor,
    verbose = verbose,
    max_iterations = max_iterations,
)


In [20]:
print(baby_agi)
print(type(baby_agi))

 '''
verbose = False 

task_list = deque([]) 

task_creation_chain = TaskCreationChain(verbose = False, 
                                        prompt = PromptTemplate(input_variables = ['incomplete_tasks', 'objective', 'result', 'task_description'], 
                                                                input_types = {}, 
                                                                partial_variables = {}, 
                                                                template = 'You are an task creation AI that uses the result of an execution agent to create new tasks with the following objective: {objective}, The last completed task has the result: {result}. This result was based on this task description: {task_description}. These are incomplete tasks: {incomplete_tasks}. Based on the result, create new tasks to be completed by the AI system that do not overlap with incomplete tasks. Return the tasks as an array.'), 
                                                                
                                        llm = OpenAI(client = <openai.resources.completions.Completions object at 0x7f139cb6cf50>, 
                                                    async_client = <openai.resources.completions.AsyncCompletions object at 0x7f13996efd40>, 
                                                    temperature = 0.0, 
                                                    model_kwargs = {}, 
                                                    openai_api_key = SecretStr('**********')), 
                                                                            
                                        output_parser = StrOutputParser(), 
                                        llm_kwargs = {}) 

task_prioritization_chain = TaskPrioritizationChain(verbose = False, 
                                                    prompt = PromptTemplate(input_variables = ['next_task_id', 'objective', 'task_names'], 
                                                                          input_types = {}, 
                                                                          partial_variables = {}, 
                                                                          template = 'You are a task prioritization AI tasked with cleaning the formatting of and reprioritizing the following tasks: {task_names}. Consider the ultimate objective of your team: {objective}. Do not remove any tasks. Return the result as a numbered list, like: #. First task #. Second task Start the task list with number {next_task_id}.'), 
                                                                          
                                                                          llm = OpenAI(client = <openai.resources.completions.Completions object at 0x7f139cb6cf50>, 
                                                                                     async_client = <openai.resources.completions.AsyncCompletions object at 0x7f13996efd40>, 
                                                                                     temperature = 0.0, model_kwargs = {}, openai_api_key = SecretStr('**********')),

                                                    output_parser = StrOutputParser(), 
                                                    llm_kwargs = {}
                                                    ) 

execution_chain = AgentExecutor(verbose = True, 
                                
                                agent = ZeroShotAgent(llm_chain = LLMChain(verbose = False, 
                                                                           prompt = PromptTemplate(input_variables = ['agent_scratchpad', 'context', 'objective', 'task'], 
                                                                                                   input_types = {}, 
                                                                                                   partial_variables = {}, 

                                                                                                   template = 'You are an AI who performs one task based on the following objective: {objective}. Take into account these previously completed tasks: {context}.\n\nSearch(query: str, **kwargs: Any) -> str - useful for when you need to answer questions about current events\nTODO(*args: Any, callbacks: Union[list[langchain_core.callbacks.base.BaseCallbackHandler], langchain_core.callbacks.base.BaseCallbackManager, NoneType] = None, tags: Optional[List[str]] = None, metadata: Optional[Dict[str, Any]] = None, **kwargs: Any) -> Any - useful for when you need to come up with todo lists. Input: an objective to create a todo list for. Output: a todo list for that objective. Please be very clear what the objective is!\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [Search, TODO]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nQuestion: {task}\n{agent_scratchpad}'), 
                                                                                                   llm = OpenAI(client = <openai.resources.completions.Completions object at 0x7f139cb6cf50>, 
                                                                                                                async_client=<openai.resources.completions.AsyncCompletions object at 0x7f13996efd40>, 
                                                                                                                temperature=0.0, 
                                                                                                                model_kwargs={}, 
                                                                                                                openai_api_key=SecretStr('**********')
                                                                                                                ), 

                                                                            output_parser=StrOutputParser(), 
                                                                            llm_kwargs = {}
                                                                            ), 
                                                        output_parser = MRKLOutputParser(), 
                                                        allowed_tools = ['Search', 'TODO']
                                                    ), 

                                tools = [Tool(name = 'Search', 
                                              description = 'useful for when you need to answer questions about current events', 
                                              func = <bound method SerpAPIWrapper.run of SerpAPIWrapper(search_engine = <class 'serpapi.google_search.GoogleSearch'>, 
                                                                                                        params = {'engine': 'google', 'google_domain': 'google.com', 'gl': 'us', 'hl': 'en'}, 
                                                                                                        serpapi_api_key = '96707de86285e4174a7f7a50e760a419114c0e8c646f32affb4beafe7e144418', 
                                                                                                        aiosession = None
                                                                                                        )
                                                        >
                                                ), 
                                            Tool(name = 'TODO', 
                                                 description = 'useful for when you need to come up with todo lists. Input: an objective to create a todo list for. Output: a todo list for that objective. Please be very clear what the objective is!', 
                                                 func = <bound method Chain.run of LLMChain(verbose = False, 
                                                                                            prompt = PromptTemplate(input_variables = ['objective'], 
                                                                                                                input_types = {}, 
                                                                                                                partial_variables = {}, 
                                                                                                                template = 'You are a planner who is an expert at coming up with a todo list for a given objective. Come up with a todo list for this objective: {objective}'), 
                                                                                                                llm = OpenAI(client = <openai.resources.completions.Completions object at 0x7f13996ae1e0>, 
                                                                                                                           async_client = <openai.resources.completions.AsyncCompletions object at 0x7f13996e8a70>, 
                                                                                                                           temperature = 0.0, 
                                                                                                                           model_kwargs = {}, 
                                                                                                                           openai_api_key = SecretStr('**********')), 
                                                                                                                           output_parser = StrOutputParser(), 
                                                                                                                           llm_kwargs = {}
                                                                                            )
                                                        >
                                                )
                                        ]
                                ) 

vectorstore = <langchain_community.vectorstores.faiss.FAISS object at 0x7f139c948770> 

max_iterations = 3
'''

verbose=False task_list=deque([]) task_creation_chain=TaskCreationChain(verbose=False, prompt=PromptTemplate(input_variables=['incomplete_tasks', 'objective', 'result', 'task_description'], input_types={}, partial_variables={}, template='You are an task creation AI that uses the result of an execution agent to create new tasks with the following objective: {objective}, The last completed task has the result: {result}. This result was based on this task description: {task_description}. These are incomplete tasks: {incomplete_tasks}. Based on the result, create new tasks to be completed by the AI system that do not overlap with incomplete tasks. Return the tasks as an array.'), llm=OpenAI(client=<openai.resources.completions.Completions object at 0x7f139cb6cf50>, async_client=<openai.resources.completions.AsyncCompletions object at 0x7f13996efd40>, temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********')), output_parser=StrOutputParser(), llm_kwargs={}) task_prioritizatio

In [16]:
baby_agi({"objective": OBJECTIVE})

/opt/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(



*****TASK LIST*****

1: Make a todo list

*****NEXT TASK*****

1: Make a todo list


> Entering new AgentExecutor chain...
Thought: I should create a todo list based on the given objective
Action: TODO
Action Input: Create a todo list for SF weather report
Observation: 

1. Research the current weather patterns in San Francisco.
2. Determine the sources for obtaining accurate weather information for the city.
3. Create a list of key weather elements to include in the report, such as temperature, precipitation, wind speed, and humidity.
4. Decide on the format of the report, whether it will be a written document, a presentation, or a combination of both.
5. Set a timeline for when the report needs to be completed and shared.
6. Gather data from reliable sources, such as the National Weather Service or local meteorologists.
7. Organize the data into a clear and easy-to-understand format.
8. Include any relevant graphics or visuals to enhance the report.
9. Proofread and edit the report 

{'objective': 'Write a weather report for SF today'}